In [460]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [461]:
engine = create_engine("sqlite:///gun_violence.sqlite")

In [462]:
Base = declarative_base()

In [463]:
class Gun_sales(Base):
    
    __tablename__ = 'gun_sales'
    extend_existing=True
    id = Column(Integer, primary_key=True, autoincrement=True)
    year = Column(Integer)
    state = Column(String(20))
    handgun = Column(Integer)
    long_gun = Column(Integer)
    other = Column(Integer)
    totals = Column(Integer)
    permit = Column(Integer)
    permit_recheck = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.state}"

In [464]:
class Gun_law_grade(Base):
    
    __tablename__ = 'gun_law_grade'
    
    id = Column(Integer, primary_key=True)
    ranking = Column(Integer)
    state = Column(String(255))
    grade = Column(String(255))
    gun_deaths = Column(Integer)
    gun_deaths_per_100k = Column(Float)
        
    def __repr__(self):
        return f"id={self.id}, name={self.state}"

In [465]:
# Other classes for other tables go here

In [466]:
Base.metadata.create_all(engine)

In [467]:
engine.table_names()

['gun_law_grade', 'gun_sales']

In [471]:
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table

populate_table(engine, Gun_sales.__table__, 'cleaned_data/gun_sale_est_1998_2017.csv')
populate_table(engine, Gun_law_grade.__table__, 'cleaned_data/gun_law_scorecard.csv')


In [473]:
#run query to test data
engine.execute("SELECT * FROM gun_sales LIMIT 1").fetchall()

[(1, 1998, 'Alabama', 10567, 25723, 0, 36290, 0, 0)]

In [474]:
engine.execute("SELECT * FROM gun_law_grade LIMIT 10").fetchall()

[(1, 50, 'Mississippi', 'F', 4, 19.8),
 (2, 48, 'Kansas', 'F', 23, 13.3),
 (3, 48, 'Missouri', 'F', 7, 18.8),
 (4, 47, 'Arizona', 'F', 16, 15.2),
 (5, 46, 'Idaho', 'F', 19, 14.6),
 (6, 45, 'Wyoming', 'F', 11, 17.5),
 (7, 44, 'Alaska', 'F', 1, 23.0),
 (8, 43, 'Louisiana', 'F', 3, 21.2),
 (9, 42, 'Kentucky', 'F', 13, 17.5),
 (10, 41, 'Vermont', 'F', 36, 11.0)]